In [1]:
import torch
import torchvision
from torch import nn
from torch.utils.data import Dataset,DataLoader
from tensorboardX import SummaryWriter
import time

In [2]:
train_data = torchvision.datasets.CIFAR10("./dataset",train=True,transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Resize((227,227)),]),download=True)
test_data = torchvision.datasets.CIFAR10("./dataset",train=False,transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Resize((227,227)),]),download=True)

0it [00:00, ?it/s]

Extracting ./dataset/cifar-10-python.tar.gz to ./dataset
Files already downloaded and verified


In [3]:
print("length of train_data",len(train_data),"\n")
print("length of test_data",len(test_data),"\n")

length of train_data 50000 

length of test_data 10000 



In [4]:
train_dataloader=DataLoader(train_data,128)
test_dataloader=DataLoader(test_data,128)

In [5]:
class cifar(nn.Module):
    def __init__(self):
        super(cifar,self).__init__()
        self.model1=nn.Sequential(
            nn.Conv2d(3,96,11,stride=4),
            nn.ReLU(),
            nn.BatchNorm2d(96),
            nn.MaxPool2d(3,stride=2),
            
            nn.Conv2d(96,256,5,stride=1,padding=2),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(3,stride=2),
            
            nn.Conv2d(256,384,3,stride=1,padding=1),
            nn.ReLU(),
            
            nn.Conv2d(384,384,3,stride=1,padding=1),
            nn.ReLU(),
            
            nn.Conv2d(384,256,3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(3,stride=2),
            nn.Flatten(),
            nn.Linear(9216,4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096,1000)
        )
    def forward(self,x):
        return self.model1(x)

在GPU上运行，只需要对
网络模型:
cifar1=cifar()
if torch.cuda.is_available():
    cifar1=cifar1.cuda()
数据（输入、标注）
if torch.cuda.is_available():
            imgs=imgs.cuda()
            target1=target1.cuda()
损失函数
if torch.cuda.is_available():
    loss=loss.cuda()
后加.cuda()

在GPU上运行的第二种方式：
#定义训练的设备
device=torch.device("cuda:0")#使用第一块GPU
print(device)
网络模型:
cifar1=cifar()

cifar1=cifar1.to(device)
数据（输入、标注）
imgs=imgs.to(device)
target1=target1.to(device)
损失函数
loss=loss.to(device)
ps:device=torch.device("cuda" if torch.cuda.is_available() else "cpu")#这样定义可以让系统自主选择CPU或者GPU

In [6]:
loss=nn.CrossEntropyLoss()
if torch.cuda.is_available():
    loss=loss.cuda()
cifar1=cifar()
if torch.cuda.is_available():
    cifar1=cifar1.cuda()
optim=torch.optim.SGD(cifar1.parameters(),lr=0.01)
write=SummaryWriter("logs_test")#tensorboard --logdir=logs_test --port=6009

In [7]:

start = time.clock()
for epoc in range(10):
    cost=0
    trzhunque=0
    tezhunque=0
    #训练步骤开始
    cifar1.train()
    for data in train_dataloader:
        imgs,target1=data
        if torch.cuda.is_available():
            imgs=imgs.cuda()
            target1=target1.cuda()
        y_hat=cifar1(imgs)
        y_hatindex=y_hat.argmax(axis=1)
        trzhunque+=((y_hatindex==target1).sum())
        res_loss=loss(y_hat,target1)
        cost=cost+res_loss
        optim.zero_grad()
        res_loss.backward()
        optim.step()
    print("train:epoc:{},cost:{}\n".format(epoc,cost))
    print("train:epoc:{},准确率:{}\n".format(epoc,trzhunque/len(train_data)))
    write.add_scalar("train_loss",cost,epoc)#将结果显示在tensorboard中
    torch.save(cifar1,"alexnet-{}".format(epoc))#保存每轮训练模型
    #测试步骤开始
    cifar1.eval()
    with torch.no_grad():
        costtest=0
        for data in test_dataloader:
            imgs,target=data
            if torch.cuda.is_available():
                imgs=imgs.cuda()
                target=target.cuda()
            y_h=cifar1(imgs)
            y_hind=y_h.argmax(axis=1)
            tezhunque+=((y_hind==target).sum())
            costtest+=loss(y_h,target)
        print("test loss epoc: {},cost: {}\n".format(epoc,costtest))
        print("test:epoc: {},准确率:{}\n".format(epoc,tezhunque/len(test_data)))
        write.add_scalar("test_loss",costtest,epoc)
        end1=time.clock()
        print("epoc:{},time: {}".format(epoc,end1-start))
end = time.clock()
write.close()
print("all-time",end-start)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


train:epoc:0,cost:895.64794921875

train:epoc:0,准确率:0.24143999814987183

test loss epoc: 0,cost: 139.61032104492188

test:epoc: 0,准确率:0.36739999055862427

epoc:0,time: 107.829687


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:41: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


train:epoc:1,cost:603.2020263671875

train:epoc:1,准确率:0.42691999673843384

test loss epoc: 1,cost: 145.42002868652344

test:epoc: 1,准确率:0.37849998474121094

epoc:1,time: 211.640009
train:epoc:2,cost:522.1573486328125

train:epoc:2,准确率:0.5130800008773804

test loss epoc: 2,cost: 97.44989776611328

test:epoc: 2,准确率:0.546999990940094

epoc:2,time: 315.83992
train:epoc:3,cost:466.2922058105469

train:epoc:3,准确率:0.5700399875640869

test loss epoc: 3,cost: 95.14239501953125

test:epoc: 3,准确率:0.573199987411499

epoc:3,time: 421.25757600000003
train:epoc:4,cost:416.8438720703125

train:epoc:4,准确率:0.6190999746322632

test loss epoc: 4,cost: 84.59368133544922

test:epoc: 4,准确率:0.6241999864578247

epoc:4,time: 524.465105
train:epoc:5,cost:374.73193359375

train:epoc:5,准确率:0.6598799824714661

test loss epoc: 5,cost: 80.14402770996094

test:epoc: 5,准确率:0.6466999650001526

epoc:5,time: 628.798001
train:epoc:6,cost:343.1278991699219

train:epoc:6,准确率:0.6897599697113037

test loss epoc: 6,cost: 71.081

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:43: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [8]:
import torchvision.models as models
alexnet_profe=models.alexnet(pretrained=True,progress=True)
if torch.cuda.is_available():
    alexnet_profe=alexnet_profe.cuda()

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [9]:
print(alexnet_profe)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [10]:
alexnet_profe.add_module("softmax",nn.Sequential(nn.ReLU(),nn.Linear(1000, 10, bias=True)))
print(alexnet_profe)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [11]:
optim_pro=torch.optim.SGD(alexnet_profe.parameters(),0.01)

In [12]:
#验证官方模型的分类准确率

start2=time.clock()

for epoc in range(10):
    tr_cost=0
    te_cost=0
    tr_acc=0
    te_acc=0
    zhong1=time.clock()
    alexnet_profe.train()
    for data in train_dataloader:
        imgs,target=data
        if torch.cuda.is_available():
            imgs=imgs.cuda()
            target=target.cuda()
        y_hat=alexnet_profe(imgs)
        res_loss=loss(y_hat,target)
        optim_pro.zero_grad()
        res_loss.backward()
        optim_pro.step()
        tr_acc+=((y_hat.argmax(1)==target).sum())
        tr_cost+=loss(y_hat,target)
        #print(y_hat.argmax(1).shape,target.shape,tr_cost,tr_acc)
    print("官方模型训练epoc{},loss： {}，准确率： {}\n".format(epoc,tr_cost,tr_acc/len(train_data)))
    zhong=time.clock()
    print("官方模型训练epoc{},time： {}\n".format(epoc,zhong-zhong1))
    alexnet_profe.eval()
    with torch.no_grad():
        for data in test_dataloader:
            imgs,target=data
            if torch.cuda.is_available():
                imgs=imgs.cuda()
                target=target.cuda()
            y_hat=alexnet_profe(imgs)
            te_acc+=((y_hat.argmax(1)==target).sum())
            te_cost+=loss(y_hat,target)
            #print(tr_cost,tr_acc)
        print("官方模型测试epoc:{},loss： {}，准确率： {}\n".format(epoc,te_cost,te_acc/len(test_data)))
end2=time.clock()
print("all time",end2-start2)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  # Remove the CWD from sys.path while we load stuff.


官方模型训练epoc0,loss： 790.6602783203125，准确率： 0.3271999955177307

官方模型训练epoc0,time： 89.97481800000014



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


官方模型测试epoc:0,loss： 102.54515075683594，准确率： 0.5284000039100647

官方模型训练epoc1,loss： 414.8916015625，准确率： 0.6255999803543091

官方模型训练epoc1,time： 86.307411

官方模型测试epoc:1,loss： 59.439292907714844，准确率： 0.7361999750137329

官方模型训练epoc2,loss： 297.9795837402344，准确率： 0.733199954032898

官方模型训练epoc2,time： 89.16901000000007

官方模型测试epoc:2,loss： 50.198585510253906，准确率： 0.7762999534606934

官方模型训练epoc3,loss： 240.70750427246094，准确率： 0.7864599823951721

官方模型训练epoc3,time： 94.07793199999992

官方模型测试epoc:3,loss： 46.77920913696289，准确率： 0.7895999550819397

官方模型训练epoc4,loss： 205.8035430908203，准确率： 0.8185799717903137

官方模型训练epoc4,time： 91.3470299999999

官方模型测试epoc:4,loss： 37.32817077636719，准确率： 0.8356999754905701

官方模型训练epoc5,loss： 182.27670288085938，准确率： 0.8388800024986267

官方模型训练epoc5,time： 87.10598800000002

官方模型测试epoc:5,loss： 34.215274810791016，准确率： 0.8504999876022339

官方模型训练epoc6,loss： 164.3952178955078，准确率： 0.8534799814224243

官方模型训练epoc6,time： 90.82460500000002

官方模型测试epoc:6,loss： 31.054176330566406，准确率： 0.85

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:40: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [13]:
!nvidia-smi

Thu Aug 12 09:01:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    38W / 250W |   3629MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------